In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import requests
import json
from api_keys import g_key
import gmaps
import glob

In [2]:
def raw_data_by_month(month):
    files = [i for i in glob.glob('../resources/twitterapi/{}*.{}'.format(month,'csv'))] 

    raw_df = pd.concat([pd.read_csv(f, parse_dates=['created_at']) for f in files ])  
    
    return raw_df


In [3]:
march_df = raw_data_by_month('03')
march_df = march_df.sort_values(by="created_at")
march_df

/Users/jameswhite/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


coordinates                created_at                          hashtags  \
72151         NaN 2020-03-01 12:00:00+00:00                               NaN   
72297         NaN 2020-03-01 12:00:00+00:00  Coronavirus pmi OPEC marketsnews   
72193         NaN 2020-03-01 12:00:00+00:00                               NaN   
72295         NaN 2020-03-01 12:00:00+00:00                               NaN   
72250         NaN 2020-03-01 12:00:00+00:00                       coronavirus   
...           ...                       ...                               ...   
39023         NaN 2020-03-25 12:56:29+00:00                               NaN   
39011         NaN 2020-03-25 12:56:29+00:00                           COVID19   
39026         NaN 2020-03-25 12:56:30+00:00                               NaN   
39003         NaN 2020-03-25 12:56:30+00:00                               NaN   
39025         NaN 2020-03-25 12:56:30+00:00                               NaN   

                                                   media  \
72151  https://twitter.com/goss_ie/status/12340859813...   
72297  https://twitter.com/Marketsdotcom/status/12340...   
72193                                                NaN   
72295                                                NaN   
72250  https://twitter.com/ArabNewsjp/status/12340859...   
...                                                  ...   
39023                                                NaN   
39011  https://twitter.com/ANI/status/124276819432812...   
39026                                                NaN   
39003                                                NaN   
39025                                                NaN   

                                                    urls  favorite_count  \
72151                             https://bit.ly/2TatY7m               0   
72297  http://live-trader.markets.com/trading-platfor...               0   
72193  https://www.independent.co.uk/independentpremi...               0   
72295                                                NaN               0   
72250                            https://arab.news/7nvsk               1   
...                                                  ...             ...   
39023                                                NaN               0   
39011                                                NaN               0   
39026                                                NaN               0   
39003                                                NaN               0   
39025                                                NaN               0   

                        id in_reply_to_screen_name  in_reply_to_status_id  \
72151  1234085981365600259                     NaN                    NaN   
72297  1234085980602277888                     NaN                    NaN   
72193  1234085981113982976                     NaN                    NaN   
72295  1234085978622582789                     NaN                    NaN   
72250  1234085979201396736                     NaN                    NaN   
...                    ...                     ...                    ...   
39023  1242797504199974912                     NaN                    NaN   
39011  1242797504304836611                     NaN                    NaN   
39026  1242797508134187011                     NaN                    NaN   
39003  1242797508134203392                     NaN                    NaN   
39025  1242797508218228741                     NaN                    NaN   

       in_reply_to_user_id  ... user_followers_count user_friends_count  \
72151                  NaN  ...                15301               2484   
72297                  NaN  ...                 7223                204   
72193                  NaN  ...                  412               3420   
72295                  NaN  ...                17127              10114   
72250                  NaN  ...                13408                 60   
...                    .

In [ ]:
april_df = raw_data_by_month('04')
april_df = april_df.sort_values(by="created_at")
april_df

In [ ]:
may_df = raw_data_by_month('05')
may_df = may_df.sort_values(by="created_at")
may_df

In [ ]:
def filter_data(raw_df):
    filtered_data = raw_df[['id', 'created_at', 'place', 'favorite_count','retweet_count']]
    filtered_data = filtered_data[raw_df['place'].notna()]
    filtered_data = filtered_data.reset_index(drop=True)
    filtered_data.rename(columns={'created_at':'Date', 'id':'Tweet ID', 'place': 'Place', 'favorite_count':'Favorite Count','retweet_count':'Retweet Count'}, inplace=True)
    filtered_data['Date'] = filtered_data['Date'].dt.strftime('%Y-%m-%d')
    filtered_data["Lat"] = ""
    filtered_data["Lng"] = ""
    filtered_data["Country"] = ""
    return filtered_data

In [ ]:
march_filter = filter_data(march_df)
march_filter

In [ ]:
april_filter = filter_data(april_df)
april_filter

In [ ]:
may_filter = filter_data(may_df)
may_filter

In [ ]:
# create a params dict that will be updated with new city each iteration
params = {"key": g_key}
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

In [ ]:
# Loop through the cities_pd and run a lat/long search for each city
def append_geocode_country(geocode_df):
    for index, row in geocode_df.iterrows():
        address = row['Place']

        # update address key value
        params['address'] = f"{address}"

        # make request
#         cities = requests.get(base_url, params=params)
#         cities = cities.json()
        resp_data = requests.get(base_url, params=params).json()
       
    
        try: 
            geocode_df[index, "Lat"] = resp_data["results"][0]["geometry"]["location"]["lat"]
            geocode_df.loc[index, "Lng"] = resp_data["results"][0]["geometry"]["location"]["lng"]
        except:
           pass
        try:
            address_components = resp_data['results'][0]['address_components']
            for i in range(0, len(address_components)):
                if (address_components[i]['types'][0]) == "country":
                    geocode_df.loc[index, 'Country'] = address_components[i]['short_name']
        except:
            pass    
    geocode_df['Covid Cases'] = ""
    geocode_df = geocode_df.dropna()
    return geocode_df

In [ ]:
march_data = append_geocode_country(march_filter)
march_data

In [ ]:
april_data = append_geocode_country(april_filter)
april_data

In [ ]:
may_data = append_geocode_country(may_filter)
may_data

In [ ]:
# map func
info_box_template = ""
def draw_map(month_df):
    month_df['Lat'] = pd.to_numeric(month_df['Lat'])
    month_df['Lng'] = pd.to_numeric(month_df['Lng'])
    month_df['Retweet Count'] = month_df['Retweet Count'].astype(float)
    month_df['Lat'] = pd.to_numeric(month_df['Lat'])
    month_df['Lng'] = pd.to_numeric(month_df['Lng'])
    month_df = month_df[month_df['Lat'].notna()]
    month_df['Retweet Count'] = month_df['Retweet Count'].astype(float)
    month_df['Favorite Count'] = month_df['Favorite Count'].astype(float)
    loc_df = month_df[["Lat", "Lng"]]
    
    gmaps.configure(api_key = g_key)
    map_figure = gmaps.figure()

    layer = gmaps.heatmap_layer(loc_df, weights = month_df['Favorite Count'], max_intensity=100, dissipating = False, point_radius = 0.5, opacity = 0.3)
    

    fig = gmaps.figure()
    markers = gmaps.marker_layer(loc_df)
    fig.add_layer(markers)
    fig
    
    
    return map_figure

In [ ]:
draw_map(march_data)

In [ ]:
draw_map(april_data)

In [ ]:
draw_map(march_data)